In [3]:
import galario as gr

In [6]:
import gradio as gr
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import AutoTokenizer
from transformers import pipeline
from transformers import AutoModelForQuestionAnswering
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch 

In [7]:
model_ckpt = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
modelST = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/107 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/477 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/133M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [8]:
#input - video link, output - full transcript
def get_transcript(link):
  print("******** Inside get_transcript ********")
  print(f"link to be extracted is : {link}")
  video_id = link.split("=")[1]
  # Handle additional query parameters such as timestamp, ...
  video_id = video_id.split("&")[0]
  print(f"video id extracted is : {video_id}")
  transcript = YouTubeTranscriptApi.get_transcript(video_id)
  FinalTranscript = ' '.join([i['text'] for i in transcript])
  return FinalTranscript,transcript, video_id
  

In [16]:
link="https://www.youtube.com/watch?v=jwtT2gC6nwE"

In [17]:
FinalTranscript,transcript, video_id=get_transcript(link)

******** Inside get_transcript ********
link to be extracted is : https://www.youtube.com/watch?v=jwtT2gC6nwE
video id extracted is : jwtT2gC6nwE


In [19]:
context = FinalTranscriptb


In [40]:
len(context)/512

17.91796875

In [20]:
question="why ae you emotional"

In [21]:
inputs = tokenizer(question, context, return_overflowing_tokens=True, max_length=512, stride = 25)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [26]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'])

In [43]:
len(inputs["input_ids"][1])

512

In [34]:
contx=[]
for window in inputs["input_ids"]:
  print(f"{tokenizer.decode(window)} \n")
  contx.append(tokenizer.decode(window).split('[SEP]')[1].strip())

[CLS] why ae you emotional [SEP] welcome back to another video and it's a special one because i'm going for pr today 3 15 is going up on the bench today that's all i gotta say so if you're hyped subscribe the channel give the video a like because it's happening why are you laughing emotional damn it do you think i'm strong because isn't your max like 275 but it's been a while 275 is my max if you can bench 315 i have to do something for you my turn [ _ _ ] we're talking about something uh oh whoa [ music ] but if you get 315 that i don't want to hear oh i got two notches if i get 315 you get the next three dinner dates you don't get it i come home with a nice surprise for me okay i know what you like you come home with a nice surprise you don't come home at all someone make pre - workout meal let me show you guys what i'm gonna make some chicken i'm gonna have rice nut butter on it we're trying to go i'm gonna peanut peanut i'm an almond [ _ _ ] you and then i'm gonna have my pre - wor

In [45]:
print(len(contx))
contx[0]

5


"welcome back to another video and it's a special one because i'm going for pr today 3 15 is going up on the bench today that's all i gotta say so if you're hyped subscribe the channel give the video a like because it's happening why are you laughing emotional damn it do you think i'm strong because isn't your max like 275 but it's been a while 275 is my max if you can bench 315 i have to do something for you my turn [ _ _ ] we're talking about something uh oh whoa [ music ] but if you get 315 that i don't want to hear oh i got two notches if i get 315 you get the next three dinner dates you don't get it i come home with a nice surprise for me okay i know what you like you come home with a nice surprise you don't come home at all someone make pre - workout meal let me show you guys what i'm gonna make some chicken i'm gonna have rice nut butter on it we're trying to go i'm gonna peanut peanut i'm an almond [ _ _ ] you and then i'm gonna have my pre - workout my wrecked i'm gonna put on

In [47]:
lst=[]
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
for contexts in contx:
 lst.append(pipe(question=question, context=contexts))

In [48]:
lst

[{'score': 0.2593109905719757,
  'start': 229,
  'end': 249,
  'answer': 'why are you laughing'},
 {'score': 0.07616408169269562,
  'start': 1494,
  'end': 1548,
  'answer': "it's been um that was a little motivation for you guys"},
 {'score': 0.0038624373264610767,
  'start': 1455,
  'end': 1466,
  'answer': "i'm filming"},
 {'score': 0.1495342254638672,
  'start': 524,
  'end': 543,
  'answer': 'thank you all right'},
 {'score': 0.003978503402322531,
  'start': 567,
  'end': 598,
  'answer': 'hope you guys enjoyed the video'}]

In [49]:
#input - question and transcript, output - answer timestamp
def get_answers_timestamp(question, final_transcript, transcript):
  print("******** Inside get_answers_timestamp ********")

  context = final_transcript
  print(f"Input Question is : {question}")
  print(f"Type of trancript is : {type(context)}, Length of transcript is : {len(context)}")
  inputs = tokenizer(question, context, return_overflowing_tokens=True, max_length=512, stride = 25)

  #getting a list of contexts available after striding
  contx=[]
  for window in inputs["input_ids"]:
      #print(f"{tokenizer.decode(window)} \n")
      contx.append(tokenizer.decode(window).split('[SEP]')[1].strip())
  #print(ques)
  #print(contx)

  lst=[]
  pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
  for contexts in contx:
    lst.append(pipe(question=question, context=contexts))
  
  print(f"contx list is : {contx}")
  lst_scores = [dicts['score'] for dicts in lst] 
  print(f"lst_scores is : {lst_scores}")
  #getting highest and second highest scores
  idxmax = lst_scores.index(max(lst_scores))
  lst_scores.remove(max(lst_scores))
  idxmax2 = lst_scores.index(max(lst_scores))
  
  sentence_for_timestamp = lst[idxmax]['answer']
  sentence_for_timestamp_secondbest = lst[idxmax2]['answer']
  
  dftranscript = pd.DataFrame(transcript)

  embedding_1= modelST.encode(dftranscript.text, convert_to_tensor=True)
  embedding_2 = modelST.encode(sentence_for_timestamp, convert_to_tensor=True)
  embedding_3 = modelST.encode(sentence_for_timestamp_secondbest, convert_to_tensor=True)
  
  similarity_tensor = util.pytorch_cos_sim(embedding_1, embedding_2)
  idx = torch.argmax(similarity_tensor)
  start_timestamp = dftranscript.iloc[[int(idx)-3]].start.values[0]
  start_timestamp = round(start_timestamp)

  similarity_tensor_secondbest = util.pytorch_cos_sim(embedding_1, embedding_3)
  idx_secondbest = torch.argmax(similarity_tensor_secondbest)
  start_timestamp_secondbest = dftranscript.iloc[[int(idx_secondbest)-3]].start.values[0]
  start_timestamp_secondbest = round(start_timestamp_secondbest)

  return start_timestamp, start_timestamp_secondbest
   

In [50]:
    
def display_vid(url, question, sample_question=None, example_video=None):
  print("******** display_vid ********")
  if question == '':
    question = sample_question
  
  #get embedding and youtube link for initial video
  html_in = "<iframe width='560' height='315' src=" + url + " frameborder='0' allowfullscreen></iframe>"
  #print(html)
  
  if len(example_video) !=0 : #is not None:
    print(f"example_video is  : {example_video}")
    url = example_video[0]
  #get transcript
  final_transcript, transcript, video_id = get_transcript(url)
  
  #get answer timestamp
  #input - question and transcript, output - answer timestamp
  ans_timestamp, ans_timestamp_secondbest = get_answers_timestamp(question, final_transcript, transcript)
  
  #created embedding  width='560' height='315' 
  html_out = "<iframe width='730' height='400' src='https://www.youtube.com/embed/" + video_id + "?start=" + str(ans_timestamp) + "' title='YouTube video player' frameborder='0' allow='accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture' allowfullscreen></iframe>"
  print(f"html output is : {html_out}")
  html_out_secondbest = "<iframe width='730' height='400' src='https://www.youtube.com/embed/" + video_id + "?start=" + str(ans_timestamp_secondbest) + "' title='YouTube video player' frameborder='0' allow='accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture' allowfullscreen></iframe>"
  
  if question == '':
    print(f"Inside display_vid(), Sample_Question coming from Radio box is BEFORE : {sample_question}")
    sample_ques = set_example_question(sample_question)
    print(f"Inside display_vid(), Sample Question coming from Radio box is AFTER : {sample_ques}")
  else:
    sample_ques = question
  return html_out, html_out_secondbest, sample_ques, url

In [51]:
def set_example_question(sample_question):
    print(f"******* Inside Sample Questions ********")
    print(f"Sample Question coming from Radio box is : {sample_question}")
    print("What is the Return value : {gr.Radio.update(value=sample_question)}")
    return gr.Radio.update(value=sample_question) #input_ques.update(example)

demo = gr.Blocks()

In [ ]:
with demo:
  gr.Markdown("<h1><center>Have you ever watched a lengthy video or podcast on YouTube and thought it would have been so much better if there had been 'explanatory' timestamps?</center></h1>")
  gr.Markdown(
        """### How many times have you seen a long video/podcast on Youtube and wondered only if there would have been 'explanatory' timestamps it would have been so much better..
            
        **Best part:** You don't even have to move away from the Space tab in your browser as the YouTube video gets played within the given View.
        """
    )
  with gr.Row():
    input_url = gr.Textbox(label="Input a Youtube video link") 
    input_ques = gr.Textbox(label="Ask a Question")

  with gr.Row():
    output_vid = gr.HTML(label="Video from timestamp 1", show_label=True)
    output_vid_secondbest = gr.HTML(label="Video from timestamp 2", show_label=True)
    
  with gr.Row():
    example_question = gr.Dropdown(
                    ["Choose a sample question", "Does video talk about different modalities", 
                    "does the model uses perceiver architecture?",
                    "when does the video talk about locked image tuning or lit?",
                    "comparison between gpt3 and jurassic?",
                    "Has flamingo passed turing test yet?",
                    "Any funny examples in video?",
                    "is it possible to download the stylegan model?",
                    "what was very cool?",
                    "what is the cool library?"], label= "Choose a sample Question", value=None)
  with gr.Row():
    example_video = gr.CheckboxGroup( ["https://www.youtube.com/watch?v=smUHQndcmOY"], label= "Choose a sample YouTube video") 
                                                                    
  b1 = gr.Button("Publish Video")
  
  b1.click(display_vid, inputs=[input_url, input_ques, example_question, example_video], outputs=[output_vid, output_vid_secondbest, input_ques, input_url])
  
  with gr.Row():
    gr.Markdown('''
    #### Model Credits
    1. [Question Answering](https://huggingface.co/deepset/minilm-uncased-squad2)
    1. [Sentence Transformer](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)
    ''')
  
  with gr.Row(): 
    gr.Markdown("![visitor badge](https://visitor-badge.glitch.me/badge?page_id=gradio-blocks_ask_questions_to_youtube_videos)")

demo.launch(enable_queue=True, debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
